<h1 align=center><font size = 5>參數的調整應用 </h1 >


<h3>Objective for this Notebook</h3>    
<h5> 1. 了解如何針對不同的Layer：載入、凍結、提取、保存權重</h5>



# Table of Contents
<li><a href="#ref0">載入權重</a></li>
<li><a href="#ref1">凍結權重</a></li>
<li><a href="#ref2">提取權重</a></li>
<li><a href="#ref3">保存權重</a></li>
<hr>


<a id="ref0"></a>
<h2 align=center>載入權重  </h2>

載入權重是將事先訓練過的模型的權重加載到當前模型中。在深度學習中，常見的做法是使用預訓練的模型進行微調，從而提高訓練效率並改善性能。預訓練模型通常是在大型數據集（如ImageNet）上訓練的，這些模型能夠學到一般性的特徵，可以作為下游任務的基礎。

導入必要套件

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
import urllib.request
import os

確認CUDA狀態

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


<h3 align=center>方法一：直接載入  </h3>

手動下載預訓練權重(以ResNet50為範例)

In [223]:
weights_url = "https://download.pytorch.org/models/resnet50-0676ba61.pth"
weights_path = "./resnet50.pth"
if not os.path.exists(weights_path):
    print("Downloading ResNet50 weights...")
    urllib.request.urlretrieve(weights_url, weights_path)
    print("Download completed!")

In [226]:
model_resnet50_case1 = models.resnet50()  # 預設載入未訓練權重的模型
model_resnet50_case1.to(device) 
pretrained_weights = torch.load(weights_path, map_location=device)
model_resnet50_case1.load_state_dict(pretrained_weights)

C:\Users\tony1\AppData\Local\Temp\ipykernel_39040\368650667.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_weights = torch.load(weights_path, map_location=de

<All keys matched successfully>

自動下載預訓練權重(以ResNet50為範例)

In [228]:
model_resnet50_case1 = models.resnet50(weights=True)

自動下載預訓練權重(以ViT為範例)

In [234]:
model_vit_b_16_case1=models.vit_b_16(weights='IMAGENET1K_V1')

<h3 align=center>方法二：Fine-Tune(CNN-based)  </h3>

載入模型

In [229]:
# 載入ResNet-50
model_resnet50_case2 = models.resnet50()
print("原始模型的架構：",model_resnet50_case2)

原始模型的架構： ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), st

將最後一個分類層的類別數量修改為5

In [230]:
in_features = model_resnet50_case2.fc.in_features  # 取得輸入特徵的數量
model_resnet50_case2.fc = nn.Linear(in_features, 5) 
print("修改後模型的輸出層：",model_resnet50_case2.fc)

修改後模型的輸出層： Linear(in_features=2048, out_features=5, bias=True)


讀取權重檔案.pth

In [231]:
# 載入預訓練權重
weights_path = "./resnet50.pth"
pretrained_weights = torch.load(weights_path, map_location=device)

C:\Users\tony1\AppData\Local\Temp\ipykernel_39040\1385590009.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_weights = torch.load(weights_path, map_location=d

不載入全連接層的權重

In [232]:
pre_dict = {k: v for k, v in pretrained_weights.items() if "fc" not in k} #同理其他層也是 if "layer4" not in k or "fc" not in k
#不熟悉語法的同學可以看一下 https://python-reference.readthedocs.io/en/latest/docs/comprehensions/dict_comprehension.html

讓模型載入權重

In [233]:
# strict=False 是確保加載的權重與模型匹配，設定為True的話必須全部參數名稱都一致
missing_keys, unexpected_keys = model_resnet50_case2.load_state_dict(pre_dict, strict=False)
#missing_keys是沒有載入的權重；unexpected_keys是預訓練權重中存在，但模型結構中未找到對應層的權重key。

print("[沒有載入的權重]:", *missing_keys,sep="\n")
print("\n")
print("[未找到對應層的權重]:", *unexpected_keys,sep="\n")



[沒有載入的權重]:
fc.weight
fc.bias


[未找到對應層的權重]:


<h3 align=center>方法二：Fine-Tune(Transformer-based)  </h3>

載入模型(以Vision Transformer為範例)
<br>
https://pytorch.org/vision/main/models/generated/torchvision.models.vit_b_16.html#torchvision.models.vit_b_16

In [235]:
model_vit_b_16_case2=models.vit_b_16(weights='IMAGENET1K_V1') #直接導入預訓練權重
pretrained_weights = model_vit_b_16_case2.state_dict()
model_vit_b_16_case2

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

將最後一個分類層的類別數量修改為5

In [236]:
in_features = model_vit_b_16_case2.heads.head.in_features  # 取得輸入特徵的數量
model_vit_b_16_case2.heads = nn.Linear(in_features, 5) #Transformer的輸出層通常為head
print("修改後模型的輸出層：",model_vit_b_16_case2.heads)

修改後模型的輸出層： Linear(in_features=768, out_features=5, bias=True)


In [237]:
pre_dict = {k: v for k, v in pretrained_weights.items() if "head" not in k}

In [238]:
missing_keys, unexpected_keys = model_vit_b_16_case2.load_state_dict(pre_dict, strict=False)

print("[沒有載入的權重]:", *missing_keys,sep="\n")
print("\n")
print("[未找到對應層的權重]:", *unexpected_keys,sep="\n")

[沒有載入的權重]:
heads.weight
heads.bias


[未找到對應層的權重]:


讀取權重檔案.pth

<a id="ref1"></a>
<h2 align=center>凍結權重  </h2>


凍結權重是指在訓練過程中將某些層的權重固定不變，使得這些層的參數不會被更新。這在微調模型時尤為重要，尤其是當您只希望對模型的部分層進行訓練時。凍結權重可以減少訓練時間，並保留預訓練模型中學到的特徵表示。

凍結權重(參數) `param.requires_grad = False`
<br>
凍結特定層的權重(參數) `model.named_parameters()`

以ResNet50為範例

In [240]:
model_resnet50_freeze = models.resnet50()

for name, param in model_resnet50_freeze.named_parameters():
    if "layer" in name : # 假設我們針對包含 "layer" 名稱的層進行凍結
        print(f"Layer: {name}, Shape: {param.shape}")
        param.requires_grad = False  # 停止更新這些層的權重
        
# 只訓練最後的分類層
model_resnet50_freeze.fc.requires_grad = True

Layer: layer1.0.conv1.weight, Shape: torch.Size([64, 64, 1, 1])
Layer: layer1.0.bn1.weight, Shape: torch.Size([64])
Layer: layer1.0.bn1.bias, Shape: torch.Size([64])
Layer: layer1.0.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Layer: layer1.0.bn2.weight, Shape: torch.Size([64])
Layer: layer1.0.bn2.bias, Shape: torch.Size([64])
Layer: layer1.0.conv3.weight, Shape: torch.Size([256, 64, 1, 1])
Layer: layer1.0.bn3.weight, Shape: torch.Size([256])
Layer: layer1.0.bn3.bias, Shape: torch.Size([256])
Layer: layer1.0.downsample.0.weight, Shape: torch.Size([256, 64, 1, 1])
Layer: layer1.0.downsample.1.weight, Shape: torch.Size([256])
Layer: layer1.0.downsample.1.bias, Shape: torch.Size([256])
Layer: layer1.1.conv1.weight, Shape: torch.Size([64, 256, 1, 1])
Layer: layer1.1.bn1.weight, Shape: torch.Size([64])
Layer: layer1.1.bn1.bias, Shape: torch.Size([64])
Layer: layer1.1.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Layer: layer1.1.bn2.weight, Shape: torch.Size([64])
Layer: layer1.1.bn2.bi

以ViT為範例

In [241]:
model_vit_b_16_freeze=models.vit_b_16(weights='IMAGENET1K_V1')

for name, param in model_vit_b_16_freeze.named_parameters():
    if "encoder" in name : # 假設我們針對包含 "encoder" 名稱的層進行凍結
        print(f"Layer: {name}, Shape: {param.shape}")
        param.requires_grad = False  # 停止更新這些層的權重

# 只訓練最後的分類層
model_vit_b_16_freeze.heads.requires_grad = True

Layer: encoder.pos_embedding, Shape: torch.Size([1, 197, 768])
Layer: encoder.layers.encoder_layer_0.ln_1.weight, Shape: torch.Size([768])
Layer: encoder.layers.encoder_layer_0.ln_1.bias, Shape: torch.Size([768])
Layer: encoder.layers.encoder_layer_0.self_attention.in_proj_weight, Shape: torch.Size([2304, 768])
Layer: encoder.layers.encoder_layer_0.self_attention.in_proj_bias, Shape: torch.Size([2304])
Layer: encoder.layers.encoder_layer_0.self_attention.out_proj.weight, Shape: torch.Size([768, 768])
Layer: encoder.layers.encoder_layer_0.self_attention.out_proj.bias, Shape: torch.Size([768])
Layer: encoder.layers.encoder_layer_0.ln_2.weight, Shape: torch.Size([768])
Layer: encoder.layers.encoder_layer_0.ln_2.bias, Shape: torch.Size([768])
Layer: encoder.layers.encoder_layer_0.mlp.0.weight, Shape: torch.Size([3072, 768])
Layer: encoder.layers.encoder_layer_0.mlp.0.bias, Shape: torch.Size([3072])
Layer: encoder.layers.encoder_layer_0.mlp.3.weight, Shape: torch.Size([768, 3072])
Layer: en

<a id="ref2"></a>
<h2 align=center>提取權重  </h2>


提取權重是指從訓練好的模型中提取某些層或參數的數值。這可以用於分析模型的內部結構，或者在不同的模型間共享權重。提取權重有助於理解模型學到的特徵，並且可以用來在不同的任務中重用這些權重。

以ResNet50為範例

In [260]:
model_resnet50_extract = models.resnet50(weights=True)

# 提取某一層的權重
conv_weights = model_resnet50_extract.conv1.weight.data

# 提取模型的所有權重
all_weights = model_resnet50_extract.state_dict()

c:\Users\tony1\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


以ViT為範例

In [261]:
model_vit_b_16_extract=models.vit_b_16(weights='IMAGENET1K_V1')

# 提取某一層的權重
attention_weights = model_vit_b_16_extract.encoder.layers[1].self_attention.in_proj_weight.data

# 提取模型的所有權重
all_weights = model_vit_b_16_extract.state_dict()

<a id="ref3"></a>
<h2 align=center>保存權重  </h2>


在深度學習中，模型的訓練過程中，權重是模型學習到的知識。保存權重的目的是將訓練好的模型狀態儲存下來，這樣你可以在以後重新加載這些權重，而不需要從頭開始訓練。這不僅有助於節省時間，還能保證訓練結果的可復現性，並能在不同的環境中進行模型部署。

以ResNet50為範例

In [264]:
model_resnet50_save = models.resnet50(weights=True)

# 保存整個模型的權重
torch.save(model_resnet50_save.state_dict(), "resnet50_weights.pth")
print("整個模型的權重已成功保存！")

# 提取並保存第一個卷積層的權重
conv1_weights = model_resnet50_save.conv1.weight.data
torch.save(conv1_weights, "resnet50_conv1_weights.pth")
print("第一個卷積層的權重已成功保存！")

# 從文件中加載權重
model_resnet50_load = models.resnet50(weights=None)
model_resnet50_load.load_state_dict(torch.load("resnet50_weights.pth"))
print("模型權重已成功加載！")

# 加載第一個卷積層的權重
model_resnet50_load.conv1.weight.data = torch.load("resnet50_conv1_weights.pth")
print("第一個卷積層的權重已成功加載！")


整個模型的權重已成功保存！
第一個卷積層的權重已成功保存！
模型權重已成功加載！
第一個卷積層的權重已成功加載！


C:\Users\tony1\AppData\Local\Temp\ipykernel_39040\3853522088.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_resnet50_load.load_state_dict(torch.load("resnet50_we

以ViT為範例

In [ ]:
import torch
from torchvision import models

# 載入ViT模型
model_vit_b_16_save = models.vit_b_16(weights='IMAGENET1K_V1')

# 保存整個模型的權重
torch.save(model_vit_b_16_save.state_dict(), "vit_b_16_weights.pth")
print("整個模型的權重已成功保存！")

# 提取並保存第二層自注意力層的權重
attention_weights = model_vit_b_16_save.encoder.layers[1].self_attention.out_proj.weight.data
torch.save(attention_weights, "vit_attention_layer_1_weights.pth")
print("第二層自注意力層的權重已成功保存！")

# 從文件中加載權重
model_vit_b_16_load = models.vit_b_16(weights=None)
model_vit_b_16_load.load_state_dict(torch.load("vit_b_16_weights.pth"))
print("模型權重已成功加載！")

# 加載第二層自注意力層的權重
model_vit_b_16_load.encoder.layers[1].self_attention.out_proj.weight.data = torch.load("vit_attention_layer_1_weights.pth")
print("第二層自注意力層的權重已成功加載！")


整個模型的權重已成功保存！
第二層自注意力層的權重已成功保存！
模型權重已成功加載！
第二層自注意力層的權重已成功加載！


C:\Users\tony1\AppData\Local\Temp\ipykernel_39040\2306062141.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_vit_b_16_load.load_state_dict(torch.load("vit_b_16_we